<a href="https://colab.research.google.com/github/Abidt2002/HeartDisease-AI-App/blob/main/Heart_Disease_App_Using_Streamlist_%26_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Install necessary packages
!pip install nbformat nbconvert nbstripout --quiet

import nbformat
from nbconvert.preprocessors import ClearOutputPreprocessor

# Path to your Heart Disease App
notebook_path = '/content/drive/MyDrive/Colab Notebooks/Heart Disease App Using Streamlist & RAG.ipynb'

# Load the notebook
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)

# Clear outputs
clear_output = ClearOutputPreprocessor()
nb, _ = clear_output.preprocess(nb, {})

# Remove widgets metadata if exists
if 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']

# Save cleaned notebook
with open(notebook_path, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print(f"✅ Notebook '{notebook_path}' is now GitHub-safe!")


✅ Notebook '/content/drive/MyDrive/Colab Notebooks/Heart Disease App Using Streamlist & RAG.ipynb' is now GitHub-safe!


In [5]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/UCI Heart Disease Data.zip"  # update path if different
extract_path = "/content/uci_heart_data"

# Extract
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Extracted files:", os.listdir(extract_path))


✅ Extracted files: ['heart_disease_uci.csv']


In [7]:
import os

# List all files inside extracted folder
for root, dirs, files in os.walk(extract_path):
    for f in files:
        print(os.path.join(root, f))


/content/uci_heart_data/heart_disease_uci.csv


In [8]:
import pandas as pd
import os

csv_file = "/content/uci_heart_data/heart_disease_uci.csv"

df = pd.read_csv(csv_file)
print("Shape:", df.shape)
df.head()


Shape: (920, 16)


,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [9]:
!pip install xgboost scikit-learn


In [10]:
import pandas as pd

# Load dataset
df = pd.read_csv("/content/uci_heart_data/heart_disease_uci.csv")

# Preview
print("Shape:", df.shape)
df.head()


Shape: (920, 16)


,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Convert target column: 0 = No disease, 1+ = Disease
df["HeartDisease"] = df["num"].apply(lambda x: 1 if x > 0 else 0)

# Features & Target
X = df.drop(["id", "dataset", "num", "HeartDisease"], axis=1)
y = df["HeartDisease"]

# Categorical & Numerical columns
cat_cols = ["sex", "cp", "restecg", "slope", "thal"]
num_cols = [col for col in X.columns if col not in cat_cols]

# Preprocessor
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])


In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [13]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Build pipeline with preprocessing + model
model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42))
])

# Train model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.8641304347826086

Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.82      0.84        82
           1       0.86      0.90      0.88       102

    accuracy                           0.86       184
   macro avg       0.86      0.86      0.86       184
weighted avg       0.86      0.86      0.86       184


Confusion Matrix:
 [[67 15]
 [10 92]]


In [14]:
import joblib

# Save model
joblib.dump(model, "/content/heart_disease_model.pkl")
print("Model saved as heart_disease_model.pkl")


Model saved as heart_disease_model.pkl


In [15]:
!pip install -q streamlit xgboost scikit-learn joblib sentence-transformers faiss-cpu PyPDF2 openai gTTS SpeechRecognition streamlit-audiorec tiktoken watchdog pyngrok
!apt-get -qq install -y ffmpeg


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.8/872.8 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.9 MB/s eta 0:00:00


In [16]:
import os
os.environ["OPENAI_API_KEY"] = "sk-REPLACE_WITH_YOUR_KEY"


In [17]:
%%writefile rag_utils.py
import os, re, faiss, numpy as np
from typing import List, Dict, Tuple
from sentence_transformers import SentenceTransformer
import PyPDF2

EMBED_MODEL_NAME = os.environ.get("EMBED_MODEL_NAME","sentence-transformers/all-MiniLM-L6-v2")
INDEX_PATH = os.environ.get("INDEX_PATH","rag_index.faiss")
STORE_PATH = os.environ.get("STORE_PATH","rag_store.npy")

def _embedder():
    return SentenceTransformer(EMBED_MODEL_NAME)

def _clean(t: str) -> str:
    t = t.replace("\n"," ").strip()
    t = re.sub(r"\s+"," ", t)
    return t

def _chunk(text: str, size=800, overlap=120):
    words = text.split()
    out = []
    i = 0
    while i < len(words):
        j = min(len(words), i+size)
        out.append(" ".join(words[i:j]))
        if j == len(words): break
        i = j - overlap
    return out

def _read(path: str) -> str:
    if path.lower().endswith(".pdf"):
        pages = []
        with open(path,"rb") as f:
            reader = PyPDF2.PdfReader(f)
            for p in reader.pages:
                pages.append(p.extract_text() or "")
        return "\n".join(pages)
    with open(path,"r",encoding="utf-8",errors="ignore") as f:
        return f.read()

def build_index(knowledge_dir="knowledge", index_path=INDEX_PATH, store_path=STORE_PATH):
    os.makedirs(knowledge_dir, exist_ok=True)
    texts, meta = [], []
    for root, _, files in os.walk(knowledge_dir):
        for fn in files:
            if fn.lower().endswith((".pdf",".txt",".md")):
                p = os.path.join(root, fn)
                raw = _clean(_read(p))
                chunks = _chunk(raw)
                for i, ch in enumerate(chunks):
                    texts.append(ch)
                    meta.append({"source": fn, "chunk": i})
    if not texts:
        raise RuntimeError("No documents found in knowledge/. Upload PDFs or TXT first.")
    model = _embedder()
    embs = model.encode(texts, convert_to_numpy=True, normalize_embeddings=True, batch_size=64)
    dim = embs.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embs)
    faiss.write_index(index, index_path)
    np.save(store_path, np.array(texts, dtype=object))
    return len(texts)

def _load(index_path=INDEX_PATH, store_path=STORE_PATH):
    if not (os.path.exists(index_path) and os.path.exists(store_path)):
        return None, None
    index = faiss.read_index(index_path)
    texts = np.load(store_path, allow_pickle=True)
    return index, texts

def search(query: str, k=5) -> List[Dict]:
    index, texts = _load()
    if index is None:
        raise RuntimeError("RAG index not found. Click 'Build/Refresh RAG index' first.")
    model = _embedder()
    q = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(q, k)
    out = []
    for s, idx in zip(D[0], I[0]):
        out.append({"score": float(s), "text": str(texts[idx])})
    return out


Writing rag_utils.py


In [18]:
%%writefile app.py
import os, json, base64, joblib
import numpy as np
import pandas as pd
import streamlit as st

from rag_utils import build_index, search

# ======= CONFIG =======
st.set_page_config(page_title="Heart Risk + LLM RAG Assistant", layout="wide")
MODEL_PATH = "/content/heart_disease_model.pkl"  # change if needed
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY","")
USE_OPENAI = bool(OPENAI_API_KEY)

# ======= LOAD MODEL (pipeline includes preprocessing) =======
@st.cache_resource
def load_model():
    return joblib.load(MODEL_PATH)
model = load_model()

# ======= LLM (OpenAI) =======
def llm_answer(prompt: str) -> str:
    if not USE_OPENAI:
        return "No API key set. General guidance: prioritize controllable risk factors (BP, cholesterol, diabetes, smoking cessation, diet, exercise). Seek clinician review for personal recommendations."
    try:
        from openai import OpenAI
        client = OpenAI(api_key=OPENAI_API_KEY)
        msg = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role":"system","content":"You are a careful, patient-friendly medical assistant. Avoid diagnosis/prescriptions; give practical, safe guidance and emphasize clinician follow-up."},
                {"role":"user","content": prompt}
            ],
            temperature=0.2,
        )
        return msg.choices[0].message.content
    except Exception as e:
        return f"(LLM error) {e}"

def build_prompt(patient_row: dict, retrieved: list, user_question: str) -> str:
    context = "\n\n".join([f"[Chunk {i+1}] {r['text']}" for i, r in enumerate(retrieved)])
    return f"""
Use ONLY the context to answer safely and clearly. Be brief and practical. No diagnosis or prescriptions.
If red-flag symptoms appear (e.g., severe chest pain, fainting, radiation to arm/jaw, shortness of breath), advise urgent medical care.

[CONTEXT]
{context}

[PATIENT DATA JSON]
{json.dumps(patient_row, ensure_ascii=False)}

[QUESTION]
{user_question}

[REQUEST]
1) Short explanation of possible meaning of risk.
2) 4-6 specific, actionable lifestyle/monitoring tips.
3) Encourage professional follow-up.
4) One-line disclaimer.
"""

# ======= TTS (gTTS) =======
def tts_to_audio_tag(text: str, out_path="response.mp3"):
    from gtts import gTTS
    tts = gTTS(text)
    tts.save(out_path)
    audio_bytes = open(out_path,"rb").read()
    b64 = base64.b64encode(audio_bytes).decode()
    return f'<audio controls autoplay src="data:audio/mp3;base64,{b64}"></audio>'

# ======= UI =======
st.title("🫀 Heart Disease Prediction + 📚 RAG Assistant (with Voice)")
st.caption("Educational tool. Not a medical diagnosis. In emergencies, call your local emergency number.")

tabs = st.tabs(["Prediction", "AI Assistant (RAG + Voice)", "Knowledge Base"])

# -------- Prediction Tab --------
with tabs[0]:
    st.subheader("Predict from CSV or Manual Entry")

    # CSV upload
    up = st.file_uploader("Upload CSV (columns should match your training data schema)", type=["csv"])
    if up:
        df = pd.read_csv(up)
    else:
        st.markdown("**Manual single-patient input**")
        # NOTE: These fields should match your training columns:
        # We trained on dataframe after dropping: id, dataset, num, HeartDisease
        # Categorical columns were handled by the pipeline (OneHot) in training stage.
        # Provide inputs that map to: age, sex, cp, trestbps, chol, fbs, restecg, thalch, exang, oldpeak, slope, ca, thal
        col1, col2, col3 = st.columns(3)
        with col1:
            age = st.number_input("age (years)", 18, 100, 55)
            sex = st.selectbox("sex", ["Male","Female"])
            cp = st.selectbox("cp", ["typical angina","atypical angina","non-anginal","asymptomatic"])
            trestbps = st.number_input("trestbps (mm Hg)", 80, 220, 130)
            chol = st.number_input("chol (mg/dl)", 80, 700, 240)
        with col2:
            fbs = st.selectbox("fbs (>120 mg/dl?)", [False, True])
            restecg = st.selectbox("restecg", ["normal","stt abnormality","lv hypertrophy"])
            thalch = st.number_input("thalch (max HR)", 60, 230, 150)
            exang = st.selectbox("exang (exercise-induced angina?)", [False, True])
            oldpeak = st.number_input("oldpeak", 0.0, 7.0, 1.0, step=0.1)
        with col3:
            slope = st.selectbox("slope", ["upsloping","flat","downsloping"])
            ca = st.selectbox("ca (0-3)", [0.0,1.0,2.0,3.0])
            thal = st.selectbox("thal", ["normal","fixed defect","reversible defect"])
        df = pd.DataFrame([{
            "age":age,"sex":sex,"cp":cp,"trestbps":trestbps,"chol":chol,"fbs":fbs,
            "restecg":restecg,"thalch":thalch,"exang":exang,"oldpeak":oldpeak,"slope":slope,
            "ca":ca,"thal":thal
        }])

    st.write("### Input Preview")
    st.dataframe(df.head())

    # Predict
    if st.button("Predict risk"):
        preds = model.predict(df)
        probs = None
        try:
            probs = model.predict_proba(df)[:,1]
        except Exception:
            pass
        out = df.copy()
        out["Prediction"] = preds.astype(int)
        if probs is not None:
            out["RiskProb"] = np.round(probs, 3)
        st.write("### Results")
        st.dataframe(out)

# -------- Assistant Tab --------
with tabs[1]:
    st.subheader("Ask the Assistant (RAG + Voice)")

    # text input
    user_q = st.text_area("Type your question (e.g., 'What lifestyle changes should I focus on with high cholesterol and chest pain?')")

    # optional: patient JSON for personalization (keys matching your schema)
    example = {"age":55,"sex":"Male","cp":"non-anginal","trestbps":138,"chol":220,"fbs":False,"restecg":"normal","thalch":145,"exang":False,"oldpeak":1.0,"slope":"flat","ca":0.0,"thal":"normal"}
    raw_json = st.text_area("Optional patient JSON to personalize advice", json.dumps(example))

    # voice input (browser recorder)
    st.write("🎤 Voice question (record in browser):")
    wav_data = None
    try:
        from st_audiorec import st_audiorec
        wav_data = st_audiorec()
    except Exception:
        st.info("Install streamlit-audiorec if you want to record voice (already added to requirements).")

    if wav_data is not None:
        with open("temp_input.wav","wb") as f:
            f.write(wav_data)
        # STT using SpeechRecognition (Google)
        import speech_recognition as sr
        r = sr.Recognizer()
        with sr.AudioFile("temp_input.wav") as src:
            audio = r.record(src)
        try:
            text = r.recognize_google(audio)
            st.success(f"Recognized: {text}")
            user_q = text
        except Exception as e:
            st.error(f"Speech recognition failed: {e}")

    # retrieve + answer
    topk = st.slider("Top-k context chunks", 2, 8, 5)
    if st.button("Search & Answer"):
        if not user_q.strip():
            st.warning("Please type or speak a question.")
        else:
            with st.spinner("Retrieving context..."):
                hits = search(user_q, k=topk)
            st.markdown("#### Retrieved context")
            for h in hits:
                st.write(f"*Score:* {h['score']:.3f}")
                st.write(h["text"][:600] + ("..." if len(h["text"])>600 else ""))
                st.divider()
            try:
                patient_row = json.loads(raw_json)
            except Exception:
                patient_row = {}
            prompt = build_prompt(patient_row, hits, user_q)
            answer = llm_answer(prompt)
            st.markdown("### 🧠 Assistant")
            st.write(answer)
            # voice out
            try:
                audio_tag = tts_to_audio_tag(answer, "resp.mp3")
                st.markdown(audio_tag, unsafe_allow_html=True)
            except Exception as e:
                st.info(f"TTS not available: {e}")
            st.caption("This content is for education only and not a medical diagnosis.")

# -------- Knowledge Base Tab --------
with tabs[2]:
    st.subheader("Knowledge Base (RAG)")
    st.write("Upload PDFs/TXT/MD, then build the index.")
    uploaded = st.file_uploader("Upload files", type=["pdf","txt","md"], accept_multiple_files=True)
    if uploaded:
        os.makedirs("knowledge", exist_ok=True)
        for f in uploaded:
            with open(os.path.join("knowledge", f.name), "wb") as out:
                out.write(f.getbuffer())
        st.success(f"Uploaded {len(uploaded)} file(s) to knowledge/")

    if st.button("Build/Refresh RAG index"):
        try:
            with st.spinner("Indexing..."):
                n = build_index("knowledge")
            st.success(f"Indexed {n} chunks. You can now use the Assistant tab.")
        except Exception as e:
            st.error(str(e))


Writing app.py


In [19]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb


--2025-10-02 16:29:56--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.9.1/cloudflared-linux-amd64.deb [following]
--2025-10-02 16:29:56--  https://github.com/cloudflare/cloudflared/releases/download/2025.9.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/d51dc916-c5d4-4cd9-9aa5-59977c4f17b9?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-02T17%3A03%3A17Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&

In [20]:
!streamlit run app.py --server.port 8501 &>/content/logs.txt &


In [ ]:
!cloudflared tunnel --url http://localhost:8501 --no-autoupdate


2025-10-02T16:29:59Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-10-02T16:29:59Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-10-02T16:30:07Z INF +--------------------------------------------------------------------------------------------+
2025-10-02T16:30:07Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-10-02T16:30:07Z INF |  https://motels-businesses-quote-blonde.trycloudflare.